In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from helper_functions import accuracy_fn
from safetensors.torch import load_model, save_model
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Data Processing

In [3]:
df = pd.read_csv("data/EMNIST/raw/emnist-balanced-train.csv")

In [4]:
train_df = df.iloc[:round(df.shape[0]*0.8)]
test_df = df.iloc[round(df.shape[0]*0.8):]

In [5]:
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, feature_columns, target_column=None, transform=None):
        self.dataframe = dataframe
        self.feature_columns = feature_columns
        self.target_column = target_column
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        reshaped_img = self.dataframe.iloc[idx][self.feature_columns].values.reshape((28, 28)).astype(np.uint8)
        pil_img = Image.fromarray(reshaped_img)
        features = data_transforms(pil_img)
        
        # Get target if specified
        if self.target_column is not None:
            target = torch.tensor(self.dataframe.iloc[idx][self.target_column]).type(torch.LongTensor).to(device)
            return features, target
        
        # Return only features if no target column is provided
        return features

In [7]:
train_data = CustomDataset(train_df, train_df.columns[1:], train_df.columns[0])
test_data = CustomDataset(test_df, test_df.columns[1:], test_df.columns[0])

In [8]:
batch_size = 16

# put custom dataset to dataloader
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [9]:
# get how many classes are there
classes = pd.unique(train_df["45"].values)
len(classes)

47

In [10]:
# check shapes of dataloader
feature, label = next(iter(test_dl))
feature.shape, label.shape

(torch.Size([16, 3, 256, 256]), torch.Size([16]))

### Load Model

In [11]:
pret = torch.hub.load('pytorch/vision:v0.10.0', "resnet18", pretrained=False)
pret.to(device)

Using cache found in C:\Users\paoma/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\paoma\miniconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\paoma\miniconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
class MNISTModel(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.output = nn.Linear(1000, 10) # output only 10 classifications

    def forward(self, x):
        x = self.pretrained(x)
        return self.output(x)

mnist = MNISTModel(pret).to(device)
load_model(mnist, "resnet_mnist.safetensors")

(set(), [])

In [13]:
class EMNISTModel(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.output = nn.Linear(10, 47) # output only 10 classifications

    def forward(self, x):
        x = self.pretrained(x)
        return self.output(x)

model = EMNISTModel(mnist).to(device)

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

### Train Model

In [15]:
torch.manual_seed(20)

epochs = 5

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n------")
    
    # TRAINING
    train_loss, train_acc = 0, 0
    model.train()
    for batch, (X, y) in enumerate(train_dl):

        X, y = X.to(device), y.to(device)

        # forward pass
        train_pred = model(X)

        # metrics
        loss = loss_fn(train_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y, y_pred=train_pred.argmax(dim=1))

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # print metrics
    train_loss /= len(train_dl)
    train_acc /= len(train_dl)
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    
    # TESTING
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in test_dl:

            X, y = X.to(device), y.to(device)


            # forward pass
            test_pred = model(X)

            # metrics
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        
        # print metrics
        test_loss /= len(test_dl)
        test_acc /= len(test_dl)
        print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
------
Train Loss: 3.4712 | Train Accuracy: 8.37%


 20%|██        | 1/5 [08:12<32:50, 492.72s/it]

Test Loss: 2.9953 | Test Accuracy: 15.16%
Epoch: 1
------
Train Loss: 2.7427 | Train Accuracy: 20.61%


 40%|████      | 2/5 [16:33<24:52, 497.62s/it]

...

Test Loss: 1.9356 | Test Accuracy: 40.29%
Epoch: 3
------
Train Loss: 1.7959 | Train Accuracy: 44.78%


 80%|████████  | 4/5 [33:04<08:15, 495.55s/it]

Test Loss: 1.5346 | Test Accuracy: 52.70%
Epoch: 4
------
Train Loss: 1.4461 | Train Accuracy: 55.44%


100%|██████████| 5/5 [41:20<00:00, 496.11s/it]

Test Loss: 1.2641 | Test Accuracy: 60.84%
